## Работа с API

Библиотеки:

* pandas для работы с данными
* matplotlib для визуализации 
* time, datetime для работы со временем и датами
* requests для HTTPS-запросов
* json для работы с содержимым запросов 
* sqlite3 для работы с базой данных SQlite

In [1]:
import requests
import json
from datetime import timedelta, date, datetime
import time
import sqlite3
import matplotlib.pyplot as plt
import pandas as pd
import time
import os
import numpy as np
import re  # Импортируем модуль re для работы с регулярными выражениями
from tqdm import tqdm


In [4]:
timestamp = 1705846200000 / 1000  # Переводим миллисекунды в секунды
dt_object = datetime.fromtimestamp(timestamp)
print(dt_object)


2024-01-21 17:10:00


# Функция которая выполняет
### 2.Считайте список всех бирж из API и получите информацию об объёмах торгов за один последний день для любой из бирж. Обратите внимание на гранулярность данных в случае выгрузки одного дня и более двух дней.
### 3.Реализуйте генератор — функцию, в которой в цикле: совершается проход по списку всех бирж (он может быть заранее получен из API и сохранён в памяти);для каждой биржи выгружается объём торгов за последний день; собирается json вида {‘exchange_name’: volume_one_day_data}, чтобы его можно было кодировать для передачи в Kafka (метод str.encode); в конце итерации выполняется конструкция yield с кодированным json в виде bytes.

In [6]:
def fetch_exchange_volumes():
    exchange_list_response = requests.get('https://api.coingecko.com/api/v3/exchanges')
    exchange_list = exchange_list_response.json()
    results = []

    for exchange in exchange_list:
        # Получаем название биржи и обрабатываем его, если есть точка
        exchange_name = exchange['name']
        if '.' in exchange_name:  # Проверяем наличие точки в имени
            processed_exchange_name = re.split(r'\.', exchange_name)[0]  # Берем только слово до точки
        else:
            processed_exchange_name = exchange_name  # В ином случае используем полное имя

        # Получаем объем торгов за последний день для данной биржи
        trade_volume_response = requests.get(f'https://api.coingecko.com/api/v3/exchanges/{processed_exchange_name}/volume_chart?days=1')
        trade_volume_data = trade_volume_response.json()

        # Проверяем, что данные успешно получены и не содержат ошибку "exchange not found"
        if 'error' not in trade_volume_data or 'exchange not found' not in trade_volume_data['error']:
            # Формируем данные для передачи в Kafka, только если получены правильные данные
            data_to_send = {processed_exchange_name: trade_volume_data}

            # Кодируем JSON и возвращаем закодированные данные в виде bytes
            encoded_data = json.dumps(data_to_send).encode('utf-8')
            yield encoded_data
        
        # Добавляем элемент в список результатов
        results.append(encoded_data)

        # Добавляем задержку 16 секунд между запросами
        time.sleep(16)
    return results
        
for data in fetch_exchange_volumes():
    print(data)

b'{"Binance": [[1705837200000.0, "152671.2545146159386031"], [1705837800000.0, "152671.704669395745003"], [1705838400000.0, "152493.8357673268837634"], [1705839000000.0, "152516.2056461966656872"], [1705839600000.0, "152420.3743468739847294"], [1705840200000.0, "152548.3850841512079189"], [1705840800000.0, "152478.5923374656069432"], [1705841400000.0, "152138.1219398777807198"], [1705842000000.0, "151835.4371874977023068"], [1705842600000.0, "151293.3342758687662888"], [1705843200000.0, "151449.4117396325060443"], [1705843800000.0, "151506.5410696009519797"], [1705844400000.0, "151311.3492246803887405"], [1705845000000.0, "151555.6278466071780914"], [1705845600000.0, "151872.0828588570600491"], [1705846200000.0, "151462.0109573422559506"], [1705846800000.0, "151275.0832292097690658"], [1705847400000.0, "151137.1915748494981156"], [1705848000000.0, "151388.1565027725854665"], [1705848600000.0, "151231.5764249790654794"], [1705849200000.0, "151273.0444564461542346"], [1705849800000.0, "1

KeyboardInterrupt: 

In [3]:
def process_data(json_data):    
    # Преобразуем байтовую JSON-строку в строку и затем в словарь
    data_str = json_data.decode('utf-8')
    data = json.loads(data_str)

    
    for exchange, trade_data in data.items():
        # Преобразуем время в формат datetime и преобразуем объем торгов в числовой формат
        formatted_data = [[pd.to_datetime(entry[0], unit='ms').strftime('%Y-%m-%d %H:%M:%S'), float(entry[1])] for entry in trade_data]

        # Создаем DataFrame и группируем данные по времени с шагом в 4 часа, вычисляем среднее и убираем последнюю дублирующуюся строку
        hourly_aggregated = pd.DataFrame(formatted_data, columns=['timestamp', 'volume'])
        hourly_aggregated['timestamp'] = pd.to_datetime(hourly_aggregated['timestamp'])
        hourly_aggregated = hourly_aggregated.set_index('timestamp')
        hourly_aggregated = hourly_aggregated.resample('4H').mean().iloc[:-1]
        hourly_aggregated = hourly_aggregated.reset_index()
        hourly_aggregated['timestamp'] = hourly_aggregated['timestamp'].astype(str)
        hourly_aggregated = hourly_aggregated.transpose()
        hourly_aggregated.reset_index(drop= True , inplace= True )
        hourly_aggregated.columns=hourly_aggregated.iloc[0]
        hourly_aggregated = hourly_aggregated.iloc [1: , :]
        hourly_aggregated.insert (loc= 0 , column='exchange', value=data.keys())
    
    return hourly_aggregated


In [3]:
import requests
import re  # Импортируем модуль re для работы с регулярными выражениями
import json
import time
import os
import pandas as pd
import numpy as np

def fetch_exchange_volumes():
    exchange_list_response = requests.get('https://api.coingecko.com/api/v3/exchanges')
    exchange_list = exchange_list_response.json()

    for exchange in exchange_list:
        # Получаем название биржи и обрабатываем его, если есть точка
        exchange_name = exchange['name']
        if '.' in exchange_name:  # Проверяем наличие точки в имени
            processed_exchange_name = re.split(r'\.', exchange_name)[0]  # Берем только слово до точки
        else:
            processed_exchange_name = exchange_name  # В ином случае используем полное имя

        # Получаем объем торгов за последний день для данной биржи
        trade_volume_response = requests.get(f'https://api.coingecko.com/api/v3/exchanges/{processed_exchange_name}/volume_chart?days=1')
        trade_volume_data = trade_volume_response.json()

        # Проверяем, что данные успешно получены и не содержат ошибку "exchange not found"
        if 'error' not in trade_volume_data or 'exchange not found' not in trade_volume_data['error']:
            # Формируем данные для передачи в Kafka, только если получены правильные данные
            data_to_send = {processed_exchange_name: trade_volume_data}

            # Кодируем JSON и возвращаем закодированные данные в виде bytes
            encoded_data = json.dumps(data_to_send).encode('utf-8')
            yield encoded_data

        # Добавляем задержку 16 секунд между запросами
        time.sleep(16)

def aggregate_trade_volumes(data, exchange_name):
    # Преобразуем объемы торгов из строкового в числовой формат
    float_volumes = [float(entry[1]) for entry in data]

    # Вычисляем среднее значение объема торгов по окну в 4 часа
    aggregated_volumes = [np.mean(float_volumes[i:i+4]) for i in range(0, len(float_volumes)-3)]

    return aggregated_volumes

def save_to_csv(exchange_data, exchange_name):
    df = pd.DataFrame(exchange_data)
    if os.path.exists('aggregated_volumes.csv'):
        existing_data = pd.read_csv('aggregated_volumes.csv', index_col=0)
        existing_data[exchange_name] = df
        existing_data.to_csv('aggregated_volumes.csv')
    else:
        df.to_csv('aggregated_volumes.csv', index_label="Timestamp")


# Пример использования полученных данных
for data in fetch_exchange_volumes():
    decoded_data = json.loads(data.decode('utf-8'))  # Декодируем данные из bytes в JSON
    for exchange, volumes in decoded_data.items():
        # Агрегируем данные для каждой биржи
        aggregated_volumes = aggregate_trade_volumes(volumes, exchange)
        print(f"Aggregated volumes for {exchange}: {aggregated_volumes}")
        # Сохраняем в CSV
        save_to_csv({exchange: aggregated_volumes}, exchange)
        print(f"Saved aggregated volumes for {exchange} to CSV")

Aggregated volumes for Binance: [164205.5650138339, 163831.51200711416, 163491.7372228966, 162949.86491900118, 162316.3605997266, 161607.18855330197, 160820.51117485727, 160288.5206752504, 159644.6191529543, 159086.3410155407, 158601.1053403735, 157992.24943908438, 157282.50015300658, 156464.04393934362, 155576.9579431367, 154577.0345750419, 153930.00729757355, 153494.72293711768, 153113.49252983893, 153004.7130058159, 152946.2474218266, 152978.33805424123, 153024.09887024245, 152997.48389714857, 152941.33282014256, 152816.66720802028, 152761.98938215847, 152669.11226676215, 152588.2501493838, 152525.5301074483, 152494.70021113718, 152490.88935367187, 152396.36842709215, 152250.13413724807, 151936.37143517745, 151679.0762857192, 151521.18106814998, 151390.15907744569, 151455.73247013026, 151561.40024993639, 151550.26772187173, 151541.20122300406, 151436.59215506466, 151315.61056604353, 151258.00193295273, 151257.49223976184, 151167.81535034618, 150979.70916225784, 150901.96480727362, 1

KeyboardInterrupt: 

Считайте список всех бирж из API и получите информацию об объёмах торгов за один последний день для любой из бирж. Обратите внимание на гранулярность данных в случае выгрузки одного дня и более двух дней.
Реализуйте генератор — функцию, в которой в цикле: 
совершается проход по списку всех бирж (он может быть заранее получен из API и сохранён в памяти);
для каждой биржи выгружается объём торгов за последний день;
собирается json вида {‘exchange_name’: volume_one_day_data}, чтобы его можно было кодировать для передачи в Kafka (метод str.encode);
в конце итерации выполняется конструкция yield с кодированным json в виде bytes.

Реализуйте функцию, которая:
принимает на вход данные об объёмах торгов одной биржи за один день;
агрегирует (np.mean) данные до окна в 4 часа (то есть будет оставаться 24 / 4 = 6 значений объёмов торгов за один день для каждой биржи);
сохраняет промежуточный результат (например, в list или pd.DataFrame) и для каждых пяти бирж делает сохранение в csv-файл с колонками [exchange, volume_1, volume_2, …, volume_6];
если файл уже существует и в нём записаны первые пять/десять (и так далее) бирж, то на каждой пятой бирже нужно прочитать существующий файл, дописать в него новые данные и сохранить.

In [8]:
def fetch_exchange_volumes():
    exchange_list_response = requests.get('https://api.coingecko.com/api/v3/exchanges')
    exchange_list = exchange_list_response.json()

    for exchange in exchange_list:
        # Получаем название биржи и обрабатываем его, если есть точка
        exchange_name = exchange['name']
        if '.' in exchange_name:  # Проверяем наличие точки в имени
            processed_exchange_name = re.split(r'\.', exchange_name)[0]  # Берем только слово до точки
        else:
            processed_exchange_name = exchange_name  # В ином случае используем полное имя

        # Получаем объем торгов за последний день для данной биржи
        trade_volume_response = requests.get(f'https://api.coingecko.com/api/v3/exchanges/{processed_exchange_name}/volume_chart?days=1')
        trade_volume_data = trade_volume_response.json()

        # Проверяем, что данные успешно получены и не содержат ошибку "exchange not found"
        if 'error' not in trade_volume_data or 'exchange not found' not in trade_volume_data['error']:
            # Формируем данные для передачи в Kafka, только если получены правильные данные
            data_to_send = {processed_exchange_name: trade_volume_data}

            # Кодируем JSON и возвращаем закодированные данные в виде bytes
            encoded_data = json.dumps(data_to_send).encode('utf-8')
            yield encoded_data

        # Добавляем задержку 16 секунд между запросами
        time.sleep(16)

def aggregate_trade_volumes(data):
    # Преобразуем байтовую JSON-строку в строку и затем в словарь
    data_str = data.decode('utf-8')
    data = json.loads(data_str)


    for  exchange_name, trade_data in data.items():
        # Преобразуем время в формат datetime и преобразуем объем торгов в числовой формат
        formatted_data = [[pd.to_datetime(entry[0], unit='ms').strftime('%Y-%m-%d %H:%M:%S'), float(entry[1])] for entry in trade_data]

        # Создаем DataFrame и группируем данные по времени с шагом в 4 часа, вычисляем среднее и убираем последнюю дублирующуюся строку
        aggregated_volumes = pd.DataFrame(formatted_data, columns=['timestamp', 'volume'])
        aggregated_volumes['timestamp'] = pd.to_datetime(aggregated_volumes['timestamp'])
        aggregated_volumes = aggregated_volumes.set_index('timestamp')
        aggregated_volumes = aggregated_volumes.resample('4H').mean().iloc[:-1]
        aggregated_volumes = aggregated_volumes.reset_index()
        aggregated_volumes['timestamp'] = aggregated_volumes['timestamp'].astype(str)
        aggregated_volumes = aggregated_volumes.transpose()
        aggregated_volumes.reset_index(drop= True , inplace= True )
        aggregated_volumes.columns=aggregated_volumes.iloc[0]
        aggregated_volumes = aggregated_volumes.iloc [1: , :]
        aggregated_volumes.insert (loc= 0 , column='exchange', value=data.keys())

    return aggregated_volumes

def save_to_csv(exchange_data):
    
    concatenated_df = pd.DataFrame()
    i = 0
    for _ in range(len(fetch_exchange_volumes())):
        df = aggregate_trade_volumes(exchange_data)    
        concatenated_df = pd.concat([concatenated_df, df_new])  # Конкатенируем df_new с существующими данными
        i += 1
        if i<5:
            continue    
        elif  os.path.exists('aggregated_volumes.csv'):
            df = pd.read_csv('aggregated_volumes.csv')
            existing_data = pd.concat([df, concatenated_df], ignore_index=True)
            existing_data.to_csv('aggregated_volumes.csv', index=False)
                
        else:
            concatenated_df.to_csv('aggregated_volumes.csv', index=False)  # Создаем новый файл CSV без индексов


# Пример использования полученных данных
i = 1
concatenated_df = pd.DataFrame()
for data in fetch_exchange_volumes(): 
    print (i)  
    df_new = aggregate_trade_volumes(data)
    concatenated_df = pd.concat([concatenated_df, df_new])  # Конкатенируем df_new с существующими данными  
    
    if i<5:
        i += 1    
        print(concatenated_df)
        print(data)
        continue
    
    elif  os.path.exists('aggregated_volumes.csv'):
            df = pd.read_csv('aggregated_volumes.csv')
            existing_data = pd.concat([df, concatenated_df], ignore_index=True)
            existing_data.to_csv('aggregated_volumes.csv', index=False)
            i = 1
            concatenated_df = pd.DataFrame()
                
    else:
        concatenated_df.to_csv('aggregated_volumes.csv', index=False)  # Создаем новый файл CSV без индексов
        i = 1
        concatenated_df = pd.DataFrame()

        
    
        

1
0 exchange 2024-01-21 08:00:00 2024-01-21 12:00:00 2024-01-21 16:00:00  \
1  Binance       152671.704669       151582.844411       152404.496541   

0 2024-01-21 20:00:00 2024-01-22 00:00:00 2024-01-22 04:00:00  
1       146288.681227       154502.879459       184901.228426  
b'{"Binance": [[1705837800000.0, "152671.704669395745003"], [1705838400000.0, "152493.8357673268837634"], [1705839000000.0, "152516.2056461966656872"], [1705839600000.0, "152420.3743468739847294"], [1705840200000.0, "152548.3850841512079189"], [1705840800000.0, "152478.5923374656069432"], [1705841400000.0, "152138.1219398777807198"], [1705842000000.0, "151835.4371874977023068"], [1705842600000.0, "151293.3342758687662888"], [1705843200000.0, "151449.4117396325060443"], [1705843800000.0, "151506.5410696009519797"], [1705844400000.0, "151311.3492246803887405"], [1705845000000.0, "151555.6278466071780914"], [1705845600000.0, "151872.0828588570600491"], [1705846200000.0, "151462.0109573422559506"], [1705846800000.0,

KeyboardInterrupt: 

In [18]:
# Пример использования полученных данных
# Создаем пустой датафрейм
concatenated_df = pd.DataFrame()
i = 0
for data in fetch_exchange_volumes():
    print(data)
    df_new = aggregate_trade_volumes(data)
    concatenated_df = pd.concat([concatenated_df, df_new])  # Конкатенируем df_new с существующими данными
    i += 1
    if i<5:
        continue
    else:
        df = pd.read_csv('aggregated_volumes.csv')
        existing_data = pd.concat([df, concatenated_df], ignore_index=True)
        existing_data.to_csv('aggregated_volumes.csv', index=False)
        break
    

,exchange,2024-01-21 04:00:00,2024-01-21 08:00:00,2024-01-21 12:00:00,2024-01-21 16:00:00,2024-01-21 20:00:00,2024-01-22 00:00:00
1,Binance,162868.290537,155649.69254,151582.844411,152404.496541,146288.681227,154502.879459
1,Bybit,136719.194519,129827.092261,127235.321002,130994.816189,127278.978617,142122.563135
1,Gate,16150.115687,15734.402177,15537.334603,15554.294927,15113.780271,15773.463751
1,KuCoin,13562.010254,13212.252548,12990.16353,12889.645478,12435.902867,12832.85554
1,Kraken,5022.058825,4873.290555,4977.844018,5505.434085,6195.922814,6236.783092


In [9]:
pd.options.display.max_rows = None
existing_data = pd.read_csv('aggregated_volumes.csv')  # Читаем существующий CSV
existing_data
        

,exchange,2024-01-21 08:00:00,2024-01-21 12:00:00,2024-01-21 16:00:00,2024-01-21 20:00:00,2024-01-22 00:00:00,2024-01-22 04:00:00
0,Binance,152671.704669,151582.844411,152404.496541,146288.681227,154502.879459,184901.228426
1,Bybit,128283.700036,127235.321002,130994.816189,127278.978617,142122.563135,182926.813393
2,Gate,15564.761833,15537.334603,15554.294927,15113.780271,15773.463751,17788.008342
3,KuCoin,13057.047553,12990.163530,12889.645478,12435.902867,12832.855540,14091.827539
4,Kraken,4836.304856,4977.844018,5505.434085,6195.922814,6236.783092,7678.517678
5,Bitstamp,1896.280999,1947.409713,1950.260237,1864.047156,1881.155704,2110.309362
6,Bitget,15025.867106,14992.126523,15061.893167,14759.672108,15504.888479,17211.833379
7,BingX,10630.832035,10551.856858,10705.679801,10367.612002,11084.120720,13212.138425
8,Fastex,797.494510,765.022394,731.622759,579.561957,680.678443,1100.896898
9,Bitfinex,3117.231665,3246.383167,3112.616094,2934.244269,2983.121333,3740.344107


In [6]:
from confluent_kafka import Producer
import json
import time

# Создаём экземпляр продьюсера Kafka
producer = Producer({'bootstrap.servers': 'your_kafka_broker:port'})

# Функция для отправки данных в Kafka
def send_data_to_kafka(data):
    producer.produce('your_topic_name', key='exchange_data', value=data)
    producer.flush()

# Пример использования генератора и отправки данных в Kafka
for data in fetch_exchange_volumes():
    send_data_to_kafka(data)
    time.sleep(15)  # Ожидание 15 секунд между итерациями


Не забудьте заменить 'your_kafka_broker:port' на адрес вашего Kafka-брокера и порт, а 'your_topic_name' на название вашей темы в Kafka.

Этот код будет отправлять полученные данные с бирж на ваш сервер Kafka с интервалом в 15 секунд между итерациями.